In [1]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir(u'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\豪雨颱風期間降雨及風速測站資料\\raw data\\')
os.getcwd()

'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\豪雨颱風期間降雨及風速測站資料\\raw data'

### 讀取原始測站txt檔案
處理column讀取後跑掉問題  
迴圈化後可以完整讀取所有檔案

### 測站資訊
先根據測站資訊選擇需要的測站
再進行氣象資料篩選(直接讀取完整氣象資料記憶體不足)

In [ ]:
#建立氣象代碼轉換字典
col_dic = {'PP01':'降水量(mm)'
,'PP02':'降水時數(hr)'
,'PS01':'測站氣壓(hPa)'
,'PS02':'海平面氣壓(hPa)'
,'RH01':'相對濕度(%)'
,'RH02':'水氣壓(hPa)'
,'ST01':'天空狀況'
,'ST02':'地面狀況'
,'ST03':'雷暴'
,'ST04':'液態降水1'
,'ST05':'液態降水2'
,'ST06':'固態降水1'
,'ST07':'固態降水2'
,'ST08':'固態降水3'
,'TX01':'氣溫(℃)'
,'TX02':'最高氣溫(℃)'
,'TX03':'最低氣溫(℃)'
,'TX04':'濕球氣溫(℃)'
,'TX05':'露點溫度(℃)'
,'WD01':'平均風風速(m/s)'
,'WD02':'平均風風向(360 degree)'
,'WD03':'最大平均風風速(m/s)'
,'WD04':'最大平均風風向(360 degree)'
,'WD05':'最大瞬間風風速(m/s)'
,'WD06':'最大瞬間風風向(360 degree)'}

In [3]:
st_info = pd.read_excel(u'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\測站資料.xlsx')
st_info['站號'] = st_info['站號'].astype(str)

In [5]:
st_info[st_info['城市']=='高雄市']

,站號,站名,海拔高度(m),經度,緯度,城市,地址,資料起始日期,撤站日期,備註,原站號,新站號
18,467440,高雄,2.3,120.315733,22.565992,高雄市,前鎮區明孝里26鄰漁港南二路2號,1931-01-01,NaT,NaN,NaN,NaN
380,C0V210,復興,734.0,120.806089,23.222403,高雄市,桃源區復興里復興巷1鄰15號,2013-04-18,NaT,原(C1V210)站因擴充為氣象站，於2013/4/18升級為(C0V210)站。,C1V210,NaN
381,C0V250,甲仙,298.0,120.591758,23.080106,高雄市,甲仙區(甲仙國民中學操場後方小山丘上),1992-01-21,NaT,本站於2013/3/13進行儀器汰換，並於2013/4/18完成驗收。,NaN,NaN
382,C0V260,月眉,126.0,120.539853,22.971003,高雄市,杉林區月眉里山仙路1號月眉農場,2013-08-01,NaT,原(C1V260)站因擴充為氣象站，於2013/8/1升級為(C0V260)站。,C1V260,NaN
383,C0V310,美濃,46.0,120.519153,22.898742,高雄市,美濃區福安里中山路二段204巷65號(美濃區福安國小後方),1992-01-22,NaT,本站於 2013/6/15 進行儀器汰換，並於2013/8/1完成驗收。,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
887,C1V440,鳳山,27.0,120.356122,22.646653,高雄市,鳳山區文山路園藝巷4號(鳳山熱帶園藝試驗分所),1992-01-22,2013-09-03,因移位及擴充為氣象，於2013/12/1升級為(C0V440)站。,NaN,C0V440
888,C1V450,鳳森,61.0,120.384457,22.537976,高雄市,小港區水庫段56地號(鳳山水庫警衛室旁),1992-01-22,2012-07-31,因移位，於2012/7/31更變為(C1V451)站。,NaN,C1V451
889,C1V451,鳳森,51.0,120.392567,22.545196,高雄市,小港區水庫段96-1地號(鳳山給水廠內),2012-07-31,2013-09-13,因擴充為氣象站，於2013/12/1升級為(C0V450)站。,C1V450,C0V450
890,C1V460,楠溪,1857.0,120.892611,23.436444,高雄市,桃源區(楠溪林道17k處愛玉工寮，車程約1小時),1993-03-01,2017-02-02,NaN,NaN,NaN


In [4]:
typhoon_info =  pd.read_excel(u'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\颱風特性.xlsx')

In [8]:
typhoon_info

,名稱(中),名稱(英),年,起月,起日,起時,終月,終日,終時,經歷時長,路徑,近台強度,近台最低氣壓,近台最大風速,近台7級風暴風半徑,近台10級風暴風半徑,警報發布報數
0,98年蓮花颱風,LINFA,2009,6,19,20,6,22,8,60,9,輕,978,28,100,NaN,13.0
1,98年莫拉克颱風,MORAKOT,2009,8,5,20,8,10,5,105,3,中,955,40,250,100.0,36.0
2,芭瑪,PARMA,2009,10,3,5,10,6,17,84,特殊,中,920,43,250,80.0,29.0
3,南修,NAMTHEUN,2010,8,30,22,8,31,20,22,NaN,輕,995,18,80,NaN,9.0
4,萊羅克,LIONROCK,2010,8,31,20,9,2,14,42,9,輕,990,23,100,NaN,15.0
5,南修及萊羅克颱風,NaN,2010,8,30,22,9,2,14,64,雙颱,NaN,990,23,100,NaN,NaN
6,99年莫蘭蒂颱風,MERANTI,2010,9,9,10,9,10,11,25,NaN,輕,990,23,100,NaN,10.0
7,凡那比颱風,FANAPI,2010,9,17,23,9,20,14,63,4,中,940,45,200,80.0,22.0
8,梅姬,MEGI,2010,10,21,2,10,23,23,69,9,中,935,48,250,100.0,24.0
9,米雷,MEARI,2011,6,23,23,6,25,14,39,NaN,輕,982,28,200,NaN,14.0


In [5]:
typhoon_climate = pd.DataFrame()
#利奇馬LEKIMA無人工測站，若需資料需要改code
for i in os.listdir('man'):
    try:
        df1 = pd.read_table('auto/'+i,encoding='big5',comment='*',delimiter='\s+')
        df2 = pd.read_table('man/'+i,encoding='big5',comment='*',delimiter='\s+')
    except:
        df1 = pd.read_table('auto/'+i,comment='*',delimiter='\s+')
        df2 = pd.read_table('man/'+i,comment='*',delimiter='\s+')
    col_names_1 = df1.columns[1:]
    col_names_2 = df2.columns[1:]
    df1.drop(df1.columns[-1:],axis=1,inplace=True)
    df2.drop(df2.columns[-1:],axis=1,inplace=True)
    df1.columns=col_names_1
    df2.columns=col_names_2
    df = pd.concat([df1,df2])
    df = df[['stno','yyyymmddhh','PS01','TX01','RH01','WD01','WD02','PP01']]
    df['雨量運作正常'] = df['PP01'].map(lambda x:1 if x>=0 else 0)
    df['stno'] = df['stno'].astype(str)
    df = df.merge(st_info[['站號','海拔高度(m)','經度','緯度','城市']],left_on='stno', right_on='站號')
    #後期資料記載錯誤，-9999記載成-999.9以此類推
    #df.replace([-9991,-9996,-9997,-9998,-9999,-999.9,-999.8,-999.7,-999.6,-999.1,-99.1], np.nan,inplace=True)
    #針對需要欄位進行調整，避免錯誤格式種類繁多
    df['PS01'] = df['PS01'].map(lambda x:np.nan if x<0 else x)
    df['TX01'] = df['TX01'].map(lambda x:np.nan if x<-20 else x)
    df['WD01'] = df['WD01'].map(lambda x:np.nan if x<0 else x)
    df['WD02'] = df['WD02'].map(lambda x:np.nan if x<0 else x)
    df['PP01'] = df['PP01'].map(lambda x:np.nan if x<0 else x)
    #可以先根據測站條件進行篩選，ex:保留一定海拔以下的測站
    df = df[df['海拔高度(m)']<500]
    climate = df.groupby('城市').agg({'PS01':'min','TX01':'mean','WD01':['mean','max'],'PP01':['sum','mean','max']})
    climate['名稱(英)'] = i.split('.')[0]
    climate['縣市'] = climate.index
    #增加各地各災害的測站數量，用於計算累積雨量
    climate = climate.join(df.drop_duplicates('stno','first').groupby('城市')['stno'].count())
    climate.rename(columns={'stno':'測站數量',('名稱(英)', ''):'名稱(英)'},inplace=True)
    climate = climate.join(df[df['雨量運作正常']==1].drop_duplicates('stno','first').groupby('城市')['stno'].count())
    climate.rename(columns={'stno':'雨量功能正常測站數量'},inplace=True)
    typhoon_climate = pd.concat([typhoon_climate,climate])

In [24]:
#莫拉克單獨拉出
df1 = pd.read_table('auto/'+'MORAKOT.txt',encoding='big5',comment='*',delimiter='\s+')
df2 = pd.read_table('man/'+'MORAKOT.txt',encoding='big5',comment='*',delimiter='\s+')
col_names_1 = df1.columns[1:]
col_names_2 = df2.columns[1:]
df1.drop(df1.columns[-1:],axis=1,inplace=True)
df2.drop(df2.columns[-1:],axis=1,inplace=True)
df1.columns=col_names_1
df2.columns=col_names_2
df = pd.concat([df1,df2])
df = df[['stno','yyyymmddhh','PS01','TX01','RH01','WD01','WD02','PP01']]
df = df.merge(st_info[['站號','海拔高度(m)']],left_on='stno', right_on='站號')
df['雨量運作正常'] = df['PP01'].map(lambda x:1 if x>=0 else 0)
df = df[df['海拔高度(m)']<500]
df['PS01'] = df['PS01'].map(lambda x:np.nan if x<0 else x)
df['TX01'] = df['TX01'].map(lambda x:np.nan if x<-20 else x)
df['WD01'] = df['WD01'].map(lambda x:np.nan if x<0 else x)
df['WD02'] = df['WD02'].map(lambda x:np.nan if x<0 else x)
df['PP01'] = df['PP01'].map(lambda x:np.nan if x<0 else x)

In [25]:
climate = df.groupby('stno').agg({'PS01':'min','TX01':'mean','WD01':['mean','max'],'PP01':['sum','mean','max']})
climate['stno'] = climate.index

In [30]:
climate.describe().T

count        mean         std         min         25%         50%  \
PS01 min    85.0  952.072941   16.733459  908.200000  942.300000  954.500000   
TX01 mean  104.0   26.719046    1.131740   24.066667   26.018307   26.906537   
WD01 mean  104.0    3.051369    2.146363    0.338028    1.617503    2.517728   
     max   104.0   10.432692    6.518539    1.800000    6.425000    9.100000   
PP01 sum   256.0  556.484375  422.360458    0.000000  223.875000  439.500000   
     mean  255.0    3.968401    2.965517    0.000000    1.582523    3.177083   
     max   255.0   48.931373   27.956311    0.000000   30.000000   44.000000   

                  75%          max  
PS01 min   966.300000   978.100000  
TX01 mean   27.533161    28.762500  
WD01 mean    3.625863    11.879861  
     max    13.450000    45.400000  
PP01 sum   789.250000  2355.500000  
     mean    5.559859    16.472028  
     max    63.250000   189.000000

In [32]:
climate.describe().T.to_excel(u'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\颱風氣象(500內敘述統計).xlsx')

In [47]:
#累積雨量
typhoon_climate['累積雨量'] = typhoon_climate.apply(lambda x:x[('PP01', 'sum')]/x['測站數量'],axis=1)

In [48]:
typhoon_climate[typhoon_climate['累積雨量'].isin(typhoon_climate['累積雨量'].nlargest(10))]

,"(PS01, min)","(TX01, mean)","(WD01, mean)","(WD01, max)","(PP01, sum)","(PP01, mean)","(PP01, max)",名稱(英),"(縣市, )",測站數量,雨量功能正常測站數量,累積雨量
城市,,,,,,,,,,,,
高雄市,986.2,26.507180,2.809424,17.7,13120.5,6.085575,125.0,FANAPI,高雄市,23,23,570.456522
南投縣,NaN,26.170714,1.086429,3.8,7640.5,5.020039,66.0,MORAKOT,南投縣,11,11,694.590909
嘉義市,967.0,27.039583,5.853472,14.9,748.6,5.803101,68.0,MORAKOT,嘉義市,1,1,748.600000
嘉義縣,938.2,25.097163,1.746763,6.5,10433.5,6.616043,136.5,MORAKOT,嘉義縣,11,11,948.500000
屏東縣,948.6,27.322638,4.338335,26.3,23536.0,6.362801,188.5,MORAKOT,屏東縣,28,27,840.571429
澎湖縣,971.7,27.552431,9.615625,25.5,1154.1,4.306343,50.6,MORAKOT,澎湖縣,2,2,577.050000
臺南市,954.5,26.778039,5.372026,21.6,28957.5,7.067977,99.5,MORAKOT,臺南市,29,29,998.534483
高雄市,977.0,26.713509,3.857018,16.4,22434.0,7.680246,103.0,MORAKOT,高雄市,21,21,1068.285714
屏東縣,952.5,25.745971,2.365930,17.5,16275.0,5.256783,122.0,NANMADOL,屏東縣,28,28,581.250000


In [ ]:
typhoon_climate

In [ ]:
#是否需要細微處理累積雨量的部分?(篩選出完全正常測站後加總?未完成)
"""
climate = climate.join(df.groupby('stno')['PP01'].count())
climate.rename(columns={'PP01':'比較1'},inplace=True)
climate = climate.join(df[df['雨量運作正常']==1].groupby('stno')['PP01'].count())
climate.rename(columns={'PP01':'比較2'},inplace=True)
climate['比較結果'] = climate.apply(lambda x:1 if x['比較1']==x['比較2'] else 0,axis=1)
climate = climate.join(df.drop_duplicates('stno','first').groupby('城市')['stno'].count())
"""

In [ ]:
"""
#路徑分布情況
typhoon_climate[typhoon_climate[('縣市', '')]=='南投縣'].groupby('路徑')['名稱(中)'].count()
"""

In [10]:
#颱風資訊從災損合併
""""
typhoon_climate = typhoon_climate.merge(
    typhoon_info[['名稱(中)','名稱(英)','年','起月','起日','起時','終月','終日','終時','路徑','近台強度']],
    on='名稱(英)', how='left')
typhoon_climate['路徑'].fillna('無資料',inplace=True)
#dummy路徑
#分為3、4(災損最大的類別)、其他、無資料
typhoon_climate['路徑類別'] = typhoon_climate['路徑'].map(lambda x:'第三類' if x==3 else 
                                                    ('第四類' if x==4 else 
                                                     ('無資料' if x=='無資料' else 
                                                      '其他')))
""""

In [49]:
typhoon_climate.to_excel(u'D:\\Terry\\工作文件\\氣象局計畫\\資料\\氣象資料\\颱風氣象(海拔500).xlsx',index=False)

### 檢查資料問題

In [104]:
typhoon_climate

,名稱(英),"(PS01, min)","(TX01, mean)","(WD01, mean)","(WD01, max)","(PP01, mean)","(PP01, max)","(縣市, )",名稱(中),年,起月,起日,起時,終月,終日,終時,路徑,近台強度
0,DUJUAN,958.9,25.290374,1.693583,15.4,1.910000,32.0,南投縣,杜鵑,2015,9,27,8,9,29,17,2,強
1,DUJUAN,980.8,25.836111,4.238889,14.4,4.138095,57.3,嘉義市,杜鵑,2015,9,27,8,9,29,17,2,強
2,DUJUAN,954.5,24.126389,2.158333,8.6,5.982394,70.0,嘉義縣,杜鵑,2015,9,27,8,9,29,17,2,強
3,DUJUAN,983.8,26.288194,11.475000,30.1,2.410938,41.5,基隆市,杜鵑,2015,9,27,8,9,29,17,2,強
4,DUJUAN,931.2,26.237532,5.588041,32.3,4.447054,335.0,宜蘭縣,杜鵑,2015,9,27,8,9,29,17,2,強
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,USAGI,949.8,27.363390,2.233405,11.0,0.040356,3.5,苗栗縣,天兔,2013,9,19,23,9,22,14,5,強
480,USAGI,989.7,26.872917,6.836458,10.3,0.060227,1.5,連江縣,天兔,2013,9,19,23,9,22,14,5,強
481,USAGI,990.3,28.297917,5.960417,9.9,0.178261,9.2,金門縣,天兔,2013,9,19,23,9,22,14,5,強
482,USAGI,NaN,27.609115,1.688204,7.0,0.065179,4.5,雲林縣,天兔,2013,9,19,23,9,22,14,5,強


In [51]:
typhoon_climate.groupby('名稱(英)').count()

,"(PS01, min)","(TX01, mean)","(WD01, mean)","(WD01, max)","(PP01, sum)","(PP01, mean)","(PP01, max)","(縣市, )",測站數量,雨量功能正常測站數量,累積雨量
名稱(英),,,,,,,,,,,
ATSANI,22,22,22,22,22,22,22,22,22,22,22
BAILU,22,22,22,22,22,22,22,22,22,22,22
BAVI,22,22,22,22,22,22,22,22,22,22,22
DANAS,22,22,22,22,22,22,22,22,22,22,22
DUJUAN,22,22,22,22,22,22,22,22,22,22,22
FANAPI,19,22,22,22,22,22,22,22,22,22,22
FUNG-WONG,21,22,22,22,22,22,22,22,22,22,22
HAGUPIT,22,22,22,22,22,22,22,22,22,22,22
HAITANG,22,22,22,22,22,22,22,22,22,22,22


In [64]:
i = os.listdir('auto')[0]
df1 = pd.read_table('auto/'+i,encoding='big5',comment='*',delimiter='\s+')
df2 = pd.read_table('man/'+i,encoding='big5',comment='*',delimiter='\s+')
col_names_1 = df1.columns[1:]
col_names_2 = df2.columns[1:]
df1.drop(df1.columns[-1:],axis=1,inplace=True)
df2.drop(df2.columns[-1:],axis=1,inplace=True)
df1.columns=col_names_1
df2.columns=col_names_2
df = pd.concat([df1,df2])
df = df[['stno','yyyymmddhh','PS01','TX01','RH01','WD01','WD02','PP01']]
df['名稱(英)'] = i.split('.')[0]
df = df.merge(st_info[['站號','海拔高度(m)','經度','緯度','城市']],left_on='stno', right_on='站號')

In [75]:
df = pd.concat([df1,df2])
df = df[['stno','yyyymmddhh','PS01','TX01','RH01','WD01','WD02','PP01']]

In [89]:
st_info[st_info['站號']=='C0G710']

,站號,站名,海拔高度(m),經度,緯度,城市,地址,地區,資料起始日期


In [84]:
df[df['stno']=='467990']

,stno,yyyymmddhh,PS01,TX01,RH01,WD01,WD02,PP01
2064,467990,2015092701,998.7,23.9,83,4.0,60.0,0.0
2065,467990,2015092702,998.6,24.1,80,4.5,50.0,0.0
2066,467990,2015092703,998.9,23.7,81,4.3,50.0,0.0
2067,467990,2015092704,998.7,23.6,81,4.0,40.0,0.0
2068,467990,2015092705,998.7,23.3,82,3.2,30.0,0.0
...,...,...,...,...,...,...,...,...
2131,467990,2015092920,1000.2,25.3,94,2.2,90.0,0.0
2132,467990,2015092921,1001.1,25.5,94,1.4,100.0,0.0
2133,467990,2015092922,1001.3,25.7,93,3.7,110.0,0.0
2134,467990,2015092923,1001.4,25.6,93,3.1,120.0,0.3


In [ ]:
df = pd.read_table(i,encoding='big5',comment='*',delimiter='\s+')
col_names = df.columns[1:]
df.drop(df.columns[-1:],axis=1,inplace=True)
df.columns=col_names
df = df[df['stno'].isin(stations)]
globals()['df_climate_'+str(county[k])] = pd.concat([globals()['df_climate_'+str(county[k])],df],ignore_index=True)